## Recurrence, Depth and High-dimensional data
# Keras ANN notebook

In this notebook we introduce the MNIST dataset, and present basic methods to setup and train a *artificial neural network (ANN)* with the Keras package.

The following elements will be presented:

* MNIST dataset
* data pre-processing: size reduction, scaling
* shallow ann: setup and training 
* loss and accuracy graphs
* optimizer options: learning rate
* object-oriented interface
* deep ann: setup and training
* generating network architecture graphs

**References:**
* THE MNIST DATABASE of handwritten digits(http://yann.lecun.com/exdb/mnist/)
* [Are we there yet?](http://rodrigob.github.io/are_we_there_yet/build/)
* [Keras](https://keras.io/): The Python Deep Learning library
* Getting started with the Keras [functional API](https://keras.io/getting-started/functional-api-guide/)
* Getting started with the Keras Sequential model ([object oriented](https://keras.io/getting-started/sequential-model-guide/))
* Keras [model visualization](https://keras.io/visualization/)
* An [overview](http://sebastianruder.com/optimizing-gradient-descent/) of gradient descent optimization algorithms
* [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/)
* colah's [blog](http://colah.github.io/)

*Please execute the cell bellow in order to initialize the notebook environment*

In [ ]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import mod3

plt.rcParams.update({'figure.figsize': (5.0, 4.0), 'lines.linewidth': 2.0})

## MNIST dataset

The MNIST dataset has the following properties:
* fixed-size images of handwritten digits
* images are size-normalized and centered
* training set of 60,000 samples, test set of 10,000 samples
* test samples are from different writters

The MNIST dataset has enough complexity to apply key machine learning concepts while not being too computationaly intensive to train. 

**EXERCISE 1**

The Keras framework provides access to several popular datasets, with the module `keras.datasets`. The MNIST dataset is loaded as follows:
```
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
```

Load the MNIST dataset and review its basic properties


**INSTRUCTIONS**
* load the MNIST dataset with `mnist.load_data()`
* print the shapes of `x_train`, `y_train`, `x_test` and `y_test`
* plot a few random samples from the training set using `plt.imshow(img, cmap=plt.cm.gray)`
* plot the distribution of pixel values in the train set

**EXPECTED OUTPUT**
```
[Dataset properties]
train set data shape: (60000, 28, 28)
train set label shape: (60000,)
test set data shape: (10000, 28, 28)
test set label shape: (10000,)

[Sample properties]
label: 5
shape: (28, 28)
min: 0
max: 255
```
<img src="fig/keras_ann_mnist.png" style="width:90%;height:90%;display:inline;margin:1px">
<img src="fig/keras_ann_mnist_hist.png" style="width:50%;height:50%;display:inline;margin:1px">

### Data pre-processing

**EXERCISE 2**

Several pre-processing steps should take place before training the MNIST dataset.

* reduce pixel count by factor of 4, in order to train the dataset on CPU
* scale pixel intensities between 0 and 1

**INSTRUCTIONS**
* drop one in every two pixels with smart indexing kung-fu 
* scale pixel intensities between 0 and 1
* print the shapes of `x_train`, `y_train`, `x_test` and `y_test`
* plot a few samples
* plot the distribution of the pixel values in the train set

**EXPECTED OUTPUT**
```
train set shape: (60000, 14, 14)
test set shape: (60000, 14, 14)
```
<img src="fig/keras_ann_mnist_small.png" style="width:90%;height:90%;display:inline;margin:1px">
<img src="fig/keras_ann_mnist_hist_scaled.png" style="width:50%;height:50%;display:inline;margin:1px">

## Shallow ANN

<img src="fig/keras_ann_shallow_schema.png" style="width:50%;height:50%;display:inline;margin:1px">

### Adapting dataset format for ANN encoding

As shown in the graph above, the inputs to the ANN are arranged as a vector of pixels, and the outputs are arranged as a vector of labels. The outputs are encoded as *1-out-of-n*, where all units are $0$ except for the unit corresponding to the class, i.e. label $2$ is encoded as $(0, 0, 1, 0, 0, 0, 0, 0, 0, 0)$.

**INSTRUCTIONS**
* use function `keras.utils.to_categorical()` to transform the labels to *1-out-of-n* encoding
* transform the samples to the required shape with smart indexing kung-fu

**EXPECTED OUTPUT**
```
train set data shape: (60000, 196)
train set label shape: (10,)
test set data shape: (10000, 196)
test set label shape: (10,)
```

### Network setup

Setting up a shallow ANN uses the following elements:

* `keras.layersInput()` sets the input
* `keras.layersDense()` adds a fully connected layer
* `keras.models.Model()` defines the ANN model
* `compile()` method of `keras.models.Model` implements the ANN in Tensorflow
* `summary()` method of `keras.models.Model` prints the ANN structure

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

n_out = 10
n_fc1 = 256

input_layer = Input(shape=(input_train_shape), name='input')
x = Dense(n_fc1, activation='sigmoid', name='fc_1')(input_layer)
output_layer = Dense(n_out, activation='sigmoid', name='output')(x)

model = Model(input_layer, output_layer)
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
model.summary()

**EXPECTED OUTPUT**
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (InputLayer)           (None, 196)               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 256)               50432     
_________________________________________________________________
output (Dense)               (None, 10)                2570      
=================================================================
Total params: 53,002.0
Trainable params: 53,002.0
Non-trainable params: 0.0
```

### Network train

The ANN is trained by calling the method `fit()` and specifying the train set. The trained model is evaluated by calling the method `evaluate()` and the specifying test set.

The default named parameters of the `fit()` method are the following:
* `batch_size=32`
* `epochs=1`
* `verbose=1`
* `shuffle=True`

In [ ]:
history = model.fit(input_train, labels_train)

evaluation = model.evaluate(input_test, labels_test, verbose=0)

print('\n[Train parameters]')
for item in history.params:
    print(item+':', history.params[item])
print('\n[Model evaluation]')
print('test', history.params['metrics'][0], evaluation[0])
print('test', history.params['metrics'][1], evaluation[1])

**EXPECTED OUTPUT**
```
Epoch 1/1
60000/60000 [==============================] - 4s - loss: 0.0864 - acc: 0.3356     

[Train parameters]
metrics: ['loss', 'acc']
samples: 60000
batch_size: 32
epochs: 1
do_validation: False
verbose: 1

[Model evaluation]
test loss 0.0709026087403
test acc 0.4896
```

### Changing default train parameters

Retrain the ANN under the following conditions:

* 5 training epochs
* batch size of 128

**INSTRUCTIONS**
* Call the `fit()` method with relevant names parameters.

**EXPECTED OUTPUT**
```
Epoch 1/5
60000/60000 [==============================] - 1s - loss: 0.0692 - acc: 0.5123     
Epoch 2/5
60000/60000 [==============================] - 1s - loss: 0.0656 - acc: 0.5555     
Epoch 3/5
60000/60000 [==============================] - 1s - loss: 0.0623 - acc: 0.5889     
Epoch 4/5
60000/60000 [==============================] - 1s - loss: 0.0594 - acc: 0.6194     
Epoch 5/5
60000/60000 [==============================] - 1s - loss: 0.0568 - acc: 0.6465     

[Train parameters]
metrics: ['loss', 'acc']
samples: 60000
batch_size: 128
epochs: 5
do_validation: False
verbose: 1

[Model evaluation]
test loss 0.0551402773798
test acc 0.6666
```

## Resetting the weights

You might have noticed that the ANN keeps updating its weights with each call to the `fit()` method.
Resetting the network requires to redefine its structure and compiling.

**INSTRUCTIONS**
* Redefine the network, compile and train
* compare the performance of 1 epoch vs 5 epochs training

### Changing the learning rate $\eta$

Changing the learning rate $eta$ requires creating an optimizer instance,
and passing it to the relevant optimizer option:
```
from keras import optimizers
sgd = optimizers.SGD(lr=2)
```

**INSTRUCTIONS**
* create an optimizer instance, set $\eta=2$, and pass it to the `compile()` method
* retrain network for 5 epochs

**EXPECTED OUTPUT**
```
Epoch 1/5
60000/60000 [==============================] - 1s - loss: 0.0311 - acc: 0.8160     
Epoch 2/5
60000/60000 [==============================] - 1s - loss: 0.0167 - acc: 0.9106     
Epoch 3/5
60000/60000 [==============================] - 1s - loss: 0.0142 - acc: 0.9233     
Epoch 4/5
60000/60000 [==============================] - 1s - loss: 0.0129 - acc: 0.9309     
Epoch 5/5
60000/60000 [==============================] - 1s - loss: 0.0122 - acc: 0.9337     

[Model evaluation]
test loss 0.0118559889253
test acc 0.9342
```

### Visualizing loss and accuracy

### Loss and accuracy visualisation (Tensorboard)

Open up a terminal and start a TensorBoard server that will read logs stored at `/tmp/autoencoder`.

`tensorboard --logdir=/tmp/ann`

This allows us to monitor training in the TensorBoard web interface at http://127.0.0.1:6006

In [ ]:
from keras.callbacks import TensorBoard

from keras import optimizers

n_out = 10
n_fc1 = 256
n_epochs = 5
n_batch_size = 128
eta = 2

input_layer = Input(shape=(input_train_shape), name='input')
x = Dense(n_fc1, activation='sigmoid', name='fc_1')(input_layer)
output_layer = Dense(n_out, activation='sigmoid', name='output')(x)

sgd = optimizers.SGD(lr=eta)

model = Model(input_layer, output_layer)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

history = model.fit(input_train, labels_train,
                    epochs=n_epochs,
                    batch_size=n_batch_size,
                    validation_data=(input_test, labels_test),
                    verbose=0,
                    callbacks=[TensorBoard(log_dir='/tmp/ann')])

evaluation = model.evaluate(input_test, labels_test, verbose=0)

print('\n[Model evaluation]')
print('test', history.params['metrics'][0], evaluation[0])
print('test', history.params['metrics'][1], evaluation[1])

**EXPECTED OUTPUT**

<img src="fig/tensorboard_ann_1.png" style="display:inline;margin:1px"><img src="fig/tensorboard_ann_2.png" style="display:inline;margin:1px">
<img src="fig/tensorboard_ann_3.png" style="display:inline;margin:1px"><img src="fig/tensorboard_ann_4.png" style="display:inline;margin:1px">

### Loss and accuracy visualisation

We also provide you with visualization code but first set the training loop correctly. The `fit()` function only trains over at least 1 epoch, and we need it to train over a fractions of epoch to visualise the intermediate results

**INSTRUCTIONS**
* use the code below to set the training loop

In [ ]:
n = 50
n_epochs = 2
n_eval = 0.2
n_chunk = int(n_eval*n)

indexes = np.arange(n)
for epoch in range(n_epochs):
    
    #  np.random.shuffle(indexes)
    
    # insert looping kung-fu here

**EXPECTED OUTPUT**
```
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
```

Once that is done, insert your loop below

**INSTRUCTIONS**
* insert your training loop below

In [ ]:
# setup network
n_out = 10
n_fc1 = 256
n_epochs = 5
n_batch_size = 128
eta = 2
n = input_train.shape[0]
n_eval = 0.2
n_chunk = int(n_eval*n)

input_layer = Input(shape=(input_train_shape), name='input')
x = Dense(n_fc1, activation='sigmoid', name='fc_1')(input_layer)
output_layer = Dense(n_out, activation='sigmoid', name='output')(x)

sgd = optimizers.SGD(lr=eta)

model = Model(input_layer, output_layer)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

evaluate_train = model.evaluate(input_train, labels_train, verbose=0)
evaluate_test = model.evaluate(input_test, labels_test, verbose=0)

loss_train = [evaluate_train[0]]
loss_test = [evaluate_test[0]]
accuracy_train = [evaluate_train[1]]
accuracy_test = [evaluate_test[1]]

x_range = [0]

fig=plt.figure(figsize=(9, 3))

gs = gridspec.GridSpec(1, 2)

fig1 = plt.subplot(gs[0])
plt.plot(x_range, loss_train, 'C0', alpha=0.8, label='loss train')
plt.plot(x_range, loss_test, 'C1', alpha=0.8, label='loss test')

plt.ylim([0, max(loss_train[0], loss_test[0])*1.1])
plt.xlim([0, n_epochs*1.1])
plt.title('Loss')
plt.xlabel('Epoch (n)')
plt.ylabel('Loss')
plt.legend()

fig2 = plt.subplot(gs[1])
plt.plot(x_range, loss_train, 'C0', alpha=0.8, label='loss train')
plt.plot(x_range, loss_test, 'C1', alpha=0.8, label='loss test')

plt.ylim([0, 1])
plt.xlim([0, n_epochs*1.1])
plt.title('Accuracy')
plt.xlabel('Epoch (n)')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
fig.show()
fig.canvas.draw()

indexes = np.arange(n)
for epoch in range(n_epochs):
    
    np.random.shuffle(indexes)

    # insert looping kung-fu here

evaluation = model.evaluate(input_test, labels_test, verbose=0)

print('\n[Model evaluation]')
print('test', history.params['metrics'][0], evaluation[0])
print('test', history.params['metrics'][1], evaluation[1])

**EXPECTED OUTPUT**
<img src="fig/ann_keras_plot_custom.png">

**EXPECTED OUTPUT**
```
[Model evaluation]
test loss 0.0247837326184
test acc 0.8787
```

### Object oriented interface

There is also an object oriented (OO) interface, as you can see in the exemple below.

In [ ]:
from keras import optimizers
from keras.models import Sequential

# setup network
n_out = 10
n_fc1 = 256
n_epochs = 5
n_batch_size = 128
eta = 2

model = Sequential()
from keras.layers import Dense, Activation

model.add(Dense(units=n_fc1, input_dim=input_train_shape[0]))
model.add(Activation('sigmoid'))
model.add(Dense(units=n_out))
model.add(Activation('sigmoid'))

sgd = optimizers.SGD(lr=eta)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])
model.summary()

## Deep ANN

<img src="fig/keras_ann_deep_schema.png" style="width:50%;height:50%;display:inline;margin:1px">

### Network setup and train

Adding more layers to the network is done by stacking layers before the output layer.

**INSTRUCTIONS**
* Add two additional fully connected layers to the ANN
* set layer size to 64
* set batch size to 32 and $\eta$ to 3
* retrain with  batch size of 128 and observe the difference in performance

**EXPECTED OUTPUT (with batch size = 32)**
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (InputLayer)           (None, 196)               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 64)                12608     
_________________________________________________________________
fc_2 (Dense)                 (None, 64)                4160      
_________________________________________________________________
fc_3 (Dense)                 (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 10)                650       
=================================================================
Total params: 21,578.0
Trainable params: 21,578.0
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/5
60000/60000 [==============================] - 3s - loss: 0.0901 - acc: 0.1224     
Epoch 2/5
60000/60000 [==============================] - 3s - loss: 0.0765 - acc: 0.3522     
Epoch 3/5
60000/60000 [==============================] - 3s - loss: 0.0396 - acc: 0.7501     
Epoch 4/5
60000/60000 [==============================] - 3s - loss: 0.0239 - acc: 0.8580     
Epoch 5/5
60000/60000 [==============================] - 3s - loss: 0.0187 - acc: 0.8847     

[Model evaluation]
test loss 0.0165417414045
test acc 0.8971
```

## Do you MNIST me?

Let's do a competition for training MNIST. Any changes are accepted under the following 

**CCNSS MNIST competition manifesto **
We abide to the the rules of training MNIST under the following conditions:
* results are reported by averaging 5 training sessions
* only SGD with momentum optimizer is accepted
* training set is reduced to the fist 20,000 samples

**INSTRUCTIONS**
* build from the code below that already implements the restrictions to the competition
* adapt as seen fit

```
sgd = optimizers.SGD(lr=eta, decay=1e-6, momentum=0.9)

input_train_comptetition = input_train[:20000]
labels_train_competition = labels_train[:20000]
```

In [ ]:
n_out = 10
n_fc1 = 256
n_epochs = 5
n_batch_size = 128
eta = 2

input_train_competition = input_train[:20000]
labels_train_competition = labels_train[:20000]

input_layer = Input(shape=(input_train_shape), name='input')
x = Dense(n_fc1, activation='sigmoid', name='fc_1')(input_layer)
output_layer = Dense(n_out, activation='sigmoid', name='output')(x)

optimizers.SGD(lr=eta, decay=1e-6, momentum=0.9)

model = Model(input_layer, output_layer)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

history = model.fit(input_train_competition , labels_train_competition,
                    epochs=n_epochs,
                    batch_size=n_batch_size)

evaluation = model.evaluate(input_test, labels_test, verbose=0)

print('\n[Model evaluation]')
print('test', history.params['metrics'][0], evaluation[0])
print('test', history.params['metrics'][1], evaluation[1])

## Generating network architecture graphs

In [ ]:
# from IPython.display import display, SVG
# from keras.utils import plot_model
# from keras.utils.vis_utils import model_to_dot

# # setup network
# n_w1 = 64
# lr = 3.

# input_img = Input(shape=(input_train_shape,), name='input')
# x = Dense(n_w1, activation='sigmoid', name='fc_1')(input_img)
# x = Dense(n_w1, activation='sigmoid', name='fc_2')(x)
# x = Dense(n_w1, activation='sigmoid', name='fc_3')(x)
# output = Dense(n_out, activation='sigmoid', name='output')(x)

# sgd = optimizers.SGD(lr=lr)
# model = Model(input_img, output)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

# # save model graph
# plot_model(model, to_file='ann_deep.png')

# # plot model graph
# display(SVG(model_to_dot(model).create(prog='dot', format='svg')))

**EXPECTED OUTPUT**
<img src="fig/ann_deep.png">

**EXTENDED EXERCISE 1**

Examine the effect of scaling the input scamples between 0 and 1, in addition to other typical pre-processing options, such as scaling the samples to $\mu=0$ and $\sigma=1$ both pixel-wise and image-wise.